In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple
plt.style.use('ggplot')

In [2]:
class DoesNotConvergeError(Exception):
    def __init__(self, result):
        self.result = result
        super(Exception, self).__init__("{}".format(result))

In [3]:
BisectionResult = namedtuple('BisectionResult', ['result', 'possible_error', 'res','iterations', 'iteration_list'])
def bisection(f, a, b, max_iterations=200, tol=1e-12):
    """
    Given a continuous function f(x) which crosses the x axis in the interval between a and b,
    this function searches for the intersection by iteratively halving the search space.
    f(a) and f(b) must have opposing signs.
    
    Returns a named tuple containing the resulting x coordinate, the maximum possible error,
    the residual, the number of iterations done, and the list of intermediate results for each iteration.
    """
    if f(a)*f(b) > 0: raise ValueError("f(a) and f(b) must not have the same sign")
    iterations = 0
    c = np.nan
    iteration_list = []
    while ((b - a) / 2) > tol and iterations < max_iterations:
        iterations+=1
        c = (a+b)/2
        iteration_list.append(c)
        if f(c)*f(a) > 0:
            a=c
        elif f(c)*f(b) > 0:
            b=c
        else: break
    possible_error=(b-a)/2
    return BisectionResult(c, possible_error, f(c),iterations, np.array(iteration_list))

In [4]:
#f = lambda k: (10+1/(2*k))*np.e**(-k)-7-1/(2*k)
f = lambda k: -(10+1/(2*k))*np.e**(-k)+7+1/(2*k)

In [5]:
xs = np.linspace(-3,3,1000)
ys = f(xs)

bisect_result = bisection(f, -1,2)

plt.plot((bisect_result.result,bisect_result.result), (-200, 100), label="minimum")
plt.plot(xs,ys, label="f(k)")

plt.title("2a) ")
plt.xlabel("k (m)")
plt.ylabel("f(k) (m)")
plt.legend()
plt.show()

In [6]:
f_fix = lambda k: k/7*(10+1/(2*k))*np.e**(-k)-1/14

In [7]:
FixedPointResult = namedtuple('FixedPointResult', ['result', 'iterations', 'iteration_list'])
def fixed_point(g, start, tol=1e-12,max_iterations=100):
    """
    Given a function g(x), finds the point g(r) = r by iteratively applying g.
    
    Not guaranteed to converge.
    
    Returns a namedtuple containing the resulting r value, the number of iterations taken,
    and the list of all intermediate iterations.
    """
    previous = start
    current = g(previous)
    iterations = 0
    iteration_list = []
    while np.abs(current - previous) > tol:  
        iterations += 1
        previous = current
        current=g(current)
        if iterations >= max_iterations or np.isnan(current):
            raise DoesNotConvergeError(FixedPointResult(current, iterations, np.array(iteration_list)))
        iteration_list.append(current)
    return FixedPointResult(current, iterations, np.array(iteration_list))

In [8]:
xs = np.linspace(0.10,0.5,1000)
ys = f_fix(xs)

point = fixed_point(f_fix, 1)
print(point)

plt.plot(xs,ys, label="f_fix(k)")
plt.plot(xs,xs, label="k")

#plt.plot((point,point), (plt.ylim(0), plt.ylim(1)), label="fixed point")

plt.title("2a) ")
plt.xlabel("k (m)")
plt.ylabel("f_fix(k) (m)")
plt.xlim(0,0.5)
plt.legend()
plt.show()

FixedPointResult(result=0.29670305321659407, iterations=68, iteration_list=array([ 0.39724103,  0.35803271,  0.3360505 ,  0.32266668,  0.31412983,
        0.30852748,  0.30478341,  0.30225113,  0.30052468,  0.29934123,
        0.29852699,  0.29796535,  0.29757728,  0.29730881,  0.29712293,
        0.29699416,  0.29690491,  0.29684304,  0.29680014,  0.29677039,
        0.29674976,  0.29673545,  0.29672553,  0.29671864,  0.29671387,
        0.29671055,  0.29670826,  0.29670666,  0.29670556,  0.29670479,
        0.29670426,  0.29670389,  0.29670363,  0.29670346,  0.29670333,
        0.29670325,  0.29670319,  0.29670315,  0.29670312,  0.2967031 ,
        0.29670308,  0.29670307,  0.29670307,  0.29670306,  0.29670306,
        0.29670306,  0.29670306,  0.29670306,  0.29670305,  0.29670305,
        0.29670305,  0.29670305,  0.29670305,  0.29670305,  0.29670305,
        0.29670305,  0.29670305,  0.29670305,  0.29670305,  0.29670305,
        0.29670305,  0.29670305,  0.29670305,  0.29670305,  0

In [9]:
def derive(f, h=1e-12):
    """Given a function f(x), returns a new function f'(x) which
    approximates the derivative of f at x"""
    def derived(x):
        return (f(x+h)-f(x))/h
    return derived


def newton(f, fprime, start, tol=1e-12):
    """Given a function f(x) and its derivative fprime(x), finds the
    point x where f(x)=0."""
    def g(x):
        return x - f(x)/fprime(x)
    return fixed_point(g, start, tol=tol)

def secant(f, start, tol=1e-12):
    """Given a function f(x), finds the point at which f(x)=0 using newton's
    method with an approximated derivative."""
    return newton(f, derive(f), start, tol=tol)

In [10]:
# Task 6
k=0.29670305
time_of_death = lambda t: 22 -37 + 0.5*t-1/(2*k) + (10 + 1/(2*k))*np.e**(-k*t)
derivative_of_death = lambda t: 0.5 + -k*(10 + 1/(2*k))*np.e**(-k*t)

newton(time_of_death, derivative_of_death, 5)
secant(time_of_death, 5)

FixedPointResult(result=-1.3324871338230972, iterations=17, iteration_list=array([-31.96056935, -28.59015488, -25.22024972, -21.84284515,
       -18.47190207, -15.12932668, -11.83152524,  -8.64029684,
        -5.70382579,  -3.30790429,  -1.84039634,  -1.37190716,
        -1.33275868,  -1.33248684,  -1.33248713,  -1.33248713,  -1.33248713]))

In [11]:
x = np.linspace(-10,5,1000)
plt.plot(x, time_of_death(x)+37)
plt.show()
plt.plot(x, derivative_of_death(x))
time_of_death(-1.3324)

-0.00040501157656080977

In [13]:
fixed_point(lambda t: time_of_death(t)+t, 1)

DoesNotConvergeError: FixedPointResult(result=5.579265880707039e+18, iterations=100, iteration_list=array([  5.39549713e+01,   6.42472717e+01,   7.96857210e+01,
         1.02843395e+02,   1.37579906e+02,   1.89684672e+02,
         2.67841822e+02,   3.85077546e+02,   5.60931132e+02,
         8.24711511e+02,   1.22038208e+03,   1.81388793e+03,
         2.70414671e+03,   4.03953488e+03,   6.04261714e+03,
         9.04724052e+03,   1.35541756e+04,   2.03145782e+04,
         3.04551821e+04,   4.56660880e+04,   6.84824468e+04,
         1.02706985e+05,   1.54043792e+05,   2.31049003e+05,
         3.46556820e+05,   5.19818545e+05,   7.79711132e+05,
         1.16955001e+06,   1.75430833e+06,   2.63144582e+06,
         3.94715204e+06,   5.92071137e+06,   8.88105037e+06,
         1.33215589e+07,   1.99823216e+07,   2.99734658e+07,
         4.49601819e+07,   6.74402562e+07,   1.01160368e+08,
         1.51740535e+08,   2.27610786e+08,   3.41416162e+08,
         5.12124226e+08,   7.68186322e+08,   1

In [29]:
def plot_convergence(result):
    errors = np.abs(result.result - result.iteration_list)
    changes = np.array([errors[i]/errors[i-1] for i in range(1, len(errors))])
    x = range(0, len(changes))
    plt.scatter(x, changes)
    print(changes)
    plt.xlabel("iterations")
    plt.ylabel("$e_n / e_{n-1}$")
    #plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
    plt.show()
    
def plot_quadratic_convergence(result):
    errors = np.abs(result.result - result.iteration_list)
    changes = np.array([errors[i]/(errors[i-1]**2) for i in range(1, len(errors))])
    x = range(0, len(changes))
    plt.scatter(x, changes)
    print(changes)
    plt.xlabel("iterations (N)")
    plt.ylabel("$e_n / e_{n-1}^2$")
    #plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
    plt.show()

In [30]:
result = bisection(time_of_death, -2,3)
plt.ylim(-1,2)
plot_convergence(result)
plt.show()

[  3.17866971e-01   7.29850733e-02   6.35071587e+00   4.21268718e-01
   3.13109119e-01   9.68873753e-02   4.66063108e+00   3.92718391e-01
   2.26823059e-01   7.04361420e-01   2.09862843e-01   1.88250847e+00
   2.34396945e-01   6.33133603e-01   2.89722735e-01   1.22578794e+00
   9.20991019e-02   3.92893459e+00   3.72739037e-01   1.58578924e-01
   1.65300412e+00   1.97520414e-01   1.03138391e+00   1.52144676e-02
   3.13634568e+01   4.84057880e-01   4.67065673e-01   4.29486733e-01
   3.35819685e-01   1.11057460e-02   4.35217391e+01   4.88511489e-01
   4.76482618e-01   4.50643777e-01   3.90476190e-01   2.19512195e-01
   7.77777778e-01   1.42857143e-01   3.00000000e+00   3.33333333e-01
   0.00000000e+00]


In [31]:
# Fixed point
# Just adding +t to both sides doesn't converge, but this does
def best_fixed_g(t):
    return np.log(1+(5-0.5*t)/(10+(1/(2*k))))/(-k)

result = fixed_point(best_fixed_g, -100)
print(result)
plt.ylim(0,0.5)
plot_convergence(result)

FixedPointResult(result=-1.3324871338231254, iterations=14, iteration_list=array([-1.74665623, -1.37247347, -1.33636841, -1.33286407, -1.33252374,
       -1.33249069, -1.33248748, -1.33248717, -1.33248714, -1.33248713,
       -1.33248713, -1.33248713, -1.33248713, -1.33248713]))
[ 0.09654592  0.09706499  0.09711547  0.09712038  0.09712085  0.09712089
  0.09712083  0.09712015  0.09711312  0.09704005  0.09630187  0.08864183
  0.        ]


In [ ]:
# Newton
result = newton(time_of_death, derivative_of_death, 3)
print(time_of_death(result.result))
print(result)
#plt.ylim(0,0.5)
plot_convergence(result)
plt.ylim(0,1)
plot_quadratic_convergence(result)

3.552713678800501e-15
FixedPointResult(result=-1.3324871338230981, iterations=8, iteration_list=array([-5.35953736, -3.06036045, -1.7317157 , -1.35737221, -1.33258855,
       -1.33248714, -1.33248713, -1.33248713]))
[  4.29066740e-01   2.31051990e-01   6.23328948e-02   4.07558159e-03
   1.66642205e-05   9.19654636e-07   0.00000000e+00]
[  1.06546161e-01   1.33720446e-01   1.56133353e-01   1.63776160e-01
   1.64307166e-01   5.44140765e+02   0.00000000e+00]

In [33]:
def plot_iterations(expected, tol, x, f, **kwargs):
    y = []
    for start in x:
        try:
            result = f(start)
            if np.abs(result.result - expected) < tol:
                y.append(result.iterations)
            else:
                y.append(np.nan)
        except ValueError as e:
            y.append(np.nan)
        except DoesNotConvergeError as e:
            y.append(np.nan)
    plt.scatter(x,y, **kwargs)

In [34]:
# Comparing the convergence rate of the different methods. 
x = np.linspace(-100, 100, 1000)
expected = -1.332487133
tol = 0.0001
#plt.plot(x, time_of_death(x))
#plt.plot(x, derivative_of_death(x))
plt.xlabel("starting guess (x)")
plt.ylabel("number of iterations")
plot_iterations(expected, tol, x, lambda start: fixed_point(best_fixed_g, start),color='r')
plot_iterations(expected, tol, x, lambda start: newton(time_of_death, derivative_of_death, start),color='b')
plt.show()
x = np.linspace(0, 5, 1000)
plt.xlabel("size of initial interval")
plt.ylabel("number of iterations")
plot_iterations(expected, tol, x, lambda start: bisection(time_of_death, expected-start/2, expected+start/2),color='g')
plt.show()

# Analysis: Fixed point converges for all (?) negative values, but stops converging at just over x=30
# Newton's method converges for values under ~6, but is slower than fixed point for values under -30ish.
# near the correct answer, newton is significantly faster
# Bisection plot shows nothing interesting, since we already know how many iterations valid inputs will require to get a given tolerance

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: invalid value encountered in log
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: overflow encountered in double_scalars
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: overflow encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
from scipy.optimize import fsolve

result, infodict, ier, mesg = fsolve(time_of_death, 3, full_output=True)
print(result)
print(infodict)
#plt.ylim(0,0.5)
#errors = np.abs(result.result - result.iteration_list)
#changes = np.array([errors[i]/errors[i-1] for i in range(1, len(errors))])
#x = range(0, len(changes))
#plt.scatter(x, changes)
#print(changes)
#plt.plot(x, np.poly1d(np.polyfit(x, changes, 1))(x))
#plt.show()

In [ ]:
axvline